# Set kernel to: `py-pythonEnv**`
If this kernel is not available, see the Kernel Installation Instructions section located in at the bottom of this Notebook. 

# C3 Data Science on the C3 AI Suite

## C3DS - DFEs \& Analytics

#### For questions, comments, or concerns, please email C3 Training Team \< c3training@c3.ai \>, discuss with your instructor, or leave your thoughts in the Google Forms at the end of this module.


## Table of Contents 

__[C3DS Data Flow Events (DFEs)](#part1)__

* [Section 1 -  DFEs](#1)
    * [A. Write TS Data Flow Events](#1A)
    * [B. Write Compound Data Flow Event](#1B)
    * [C. Load DFE](#1C)
    * [D. Understanding "Interval" vs. "Period"](#1D)
    
__[C3DS Analytics](#part2)__

* [Section 2 - Analytic Container Logic](#2)
    * [A. `LoadContextAll` Function](#2A)
    * [B. Run `LoadContextAll` Function](#2B)
    * [C. `ProcessSource` Function](#2C)
    * [D. Run `ProcessSource` Function](#2D)  
    
    
* [Section 3 - Executing Your Analytic](#3)   


* [Section 4 - Streaming Predictions](#4)   

[Appendix - Kernel Installation Instructions](#apdx)


# C3DS Data Flow Events (DFEs)<a class="anchor" id="part1">
## Exercise Goal

In this exercise, you will become more familiar with the **C3 Analyic Container Engine**, or **ACE**. In doing so, you will implement **Time Series Data Flow Events** (`TSDFEs`), **Compound Data Flow Events**, and manage handling these various DFEs with **Analytics**!
    
## Section 1 - DFEs <a class="anchor" id="1"></a>

A trained machine learning model can be hooked up to streaming capabilities using the **C3 Analytic Container Engine**, such that when new data arrives, "event-listeners" (called **C3 Data Flow Events**) will trigger business logic to fetch the latest persisted model, and write predictions based on the new data. Using the C3 Analytic Container Engine, failure predictions for each day can be processed and persisted for all `SourceType` instances.

The screenshot below displays the code **you will be implementing and testing for this assignment**.


The code below has been simplified for clarity.
    
<img src="http://learnc3.ai/wp-content/uploads/2020/01/ACE2.png" width="500"/>


### A. Write TS Data Flow Events <a class="anchor" id="1A">

Begin by writing one `TS Data Flow Event` file for **each** metric that you used in your trained model. Save all the files as `.c3typ` files in a directory name of your choice under the `src` directory.

**Follow the screenshot above!**

### B. Write Compound Data Flow Event <a class="anchor" id="1B">

Write a **Compound DFE** that combines together **all** your TS Data Flow Events. Suggestion would be to name your Compound DFE:  `SmartBulbPredictionAnalyticInput` and saving it as `SmartBulbPredictionAnalyticInput.c3typ`

**Follow the screenshot above!**

### C. Load DFE <a class="anchor" id="1C">

Now, provision your code into your environment, and test out your new code by **loading your DFE** in the JavaScript static console. This is the input that is passed to your Analytic Container logic. Try calling `loadDFE` on both a TS and your Compound DFE. Inspect the result. 

Look up documentation on `loadDFE` by calling `c3ShowType(AnalyticsContainer)` and finding the `loadDFE` method. As a hint, the syntax of the command will be: 

```js
var input = AnalyticsContainer.loadDFE("YOUR_DFE_NAME", 
{id: "A SOURCE INSTANCE ID", typeName: "SmartBulb"}, 
"2012-01-01")
```

Run an EvalMetric call using start '2012-01-01' and end '2012-01-02' with the following command in console:
```js
c3Viz(SmartBulb.evalMetrics({
     expressions: [YOUR METRICS],
     ids: ["YOUR SOURCE INSTANCE ID"],
     start: "2012-01-01",
     end: "2012-01-02",
     interval: "DAY"}))```

The results should be the same!
> Hint: If they aren't, you may need to change **TenantConfig** `ACE-StartofTime` in case when the ACE starts later than 2012-01-01. And make sure the interval is right.

### D. Understanding "Interval" vs. "Period" <a class="anchor" id="1D">

Change the **Period** in the `@DFE` annotation for **ALL** TS and Compound DFEs from "DAY" to "5Day". (Remember that the `TSDFE`s and `CompoundDFE`s must have the same interval and period specified.) Provision and call `loadDFE` for a simple metric and compound metric again. Inspect the result. **What has changed?** We recommend logging it in your notes. 

Now change the **Interval** from **"DAY"** to **"HOUR"** and again provision and load your DFE again to inspect the result. **What has changed?** Again, we recommend logging it in your notes. 

`DFE's granualarity changed.`

Be sure to change the **Interval** and **Period** back to match your machine learning model inputs which should be `interval` of "DAY" and `period` of "DAY".

# C3DS Analytics<a class="anchor" id="part2">
    
## Section 2 - Analytic Container Logic <a class="anchor" id="2"></a>

Begin by creating a `c3typ` file named `SmartBulbPredictionAnalytic.c3typ` in the `src` directory with the following contents:

```
type SmartBulbPredictionAnalytic mixes Analytic<SmartBulbPredictionAnalyticInput, Void> {
    loadContextAll: ~ js server
    processSource:  ~ js server
}
```

**Important Side Note**: 
- Ordinarily, instead of using `Void` in `Analytic<SmartBulbPredictionAnalyticInput, Void>`, you would specify the Analytic's output type, (in this case, `SmartBulbPrediction`). So the definition of `SmartBulbPredictionAnalytic` would be: `type SmartBulbPredictionAnalytic mixes Analytic<SmartBulbPredictionAnalyticInput, SmartBulbPrediction>`. Then, in the Analytic's `processSource` function, we would simply return the `SmartBulbPrediction`s instead of calling `mergeBatch()` on them, and ACE would handle upserting to the C3 platform for us. However, in our example, since we are not returning anything, but are instead calling `mergeBatch()` directly, we are using `Void` in place of the return type. 

We will be overwriting the `loadContextAll` and `processSource` functions in the next steps.

Now, create a file `SmartBulbPredictionAnalytic.js` in the same directory and start coding up the next sections.

**IMPORTANT NOTE**: As you code up these sections, try to run the various code blocks directly in console using Chrome Developer Tool to understand what each code block is doing. You may or may not be tested on this.

### A. `LoadContextAll` Function <a class="anchor" id="2A">

In the `loadContextAll` function, you'll want to call any data structures that will be shared in common across all your sources in order to reduce I/O. In this case, we will use the same model for all instances of SmartBulb, so it would be wise to have your LoadContextAll function return your model to be used across multiple calls of the `processSource` function. 

Note that a `fetch` call returns a `FetchResult` type, which is why we need to call `.objs` to convert to an array, and then index into the array to retrieve the correct element. 

### B. Run `LoadContextAll` Function<a class="anchor" id="2B">

Now let's test-run your `LoadContextAll` function to ensure it behaves correctly. First, we will test run the code by copying and pasting into Chrome **snippets**. Then, we will provision the code and ensure it properly runs in the C3 platform. 

#### Method 1:
In Chrome developer tools, navigate to "Sources" and select "+ New Snippet" to create a new file. Copy and paste your LoadContextAll() function. Be sure to add code that calls this function after you've pasted it and assign the return object to a variable so you can inspect it. Then click on the bottom right corner "Enter" to execute the cell. Run `c3Grid(result)` on the result to inspect whether you obtained the correct model. 

*Debugging Hint: If you have trouble, you can set breakpoints in the Snippets file by clicking on the line numbers.*

<img src='http://learnc3.ai/wp-content/uploads/2020/01/method1.png' width=600/> 

If all, looks good, move on to the next test run method. 


#### Method 2:
Provision your code to your environment. After that, in Chrome static console, call your function by entering:

```js
var context = SmartBulbPredictionAnalytic.loadContextAll()
```
Inspect the context. If the results look good **and** you don't get any errors (i.e., not warmings), move on to implementing the `processSource` function. 

> Debugging Hint: You can still set use the Chrome debugger and step through Javascript code that is provisioned to the platform! Use the "Breakpoints" tool to add a break on your function as shown in the screenshot below. When you call your function in the console, the tool will stop before executing the function and you will need to click on the button "step into function call" on the upper right corner **twice** (the one with the down arrow) before being able to step into your Analytic code. Then, you can set a breakpoint on the lines of interest as usual!

<img src='http://learnc3.ai/wp-content/uploads/2020/01/breakpoints1.png' width=500> 

<img src='http://learnc3.ai/wp-content/uploads/2020/01/breakpoints2.png' width=500>

<img src='http://learnc3.ai/wp-content/uploads/2020/01/breakpoints3.png' width=500>

<img src='http://learnc3.ai/wp-content/uploads/2020/01/breakpoints4.png' width=900>

### C. `processSource` Function <a class="anchor" id="2C">

Now implement the `processSource` function by adapting the Oil & Gas code above. Your function will be very similar but you will need to use the `SmartBulbPrediction` type and adapt some of the code to suit that type.

### D. Run `processSource` Function <a class="anchor" id="2D">

Now let's go through the same 2 steps as with LoadContextAll() to test run your function. 

#### Method 1: 
Begin by copying and pasting your code into a snippet. You'll need to separately load each of the input parameters of the `processSource` function by calling the following: 

```js
var context = SmartBulbPredictionAnalytic.loadContextAll();
var source = SmartBulb.get('YOUR SOURCE INSTANCE ID');

var input = AnalyticsContainer.loadDFE("YOUR_DFE_NAME", 
{id: "A SOURCE INSTANCE ID", typeName: "SmartBulb"}, 
"2012-01-01");
inputs = SmartBulbPredictionAnalyticInput.array([input])
```
Then pass in `source`, `inputs`, and `context` into your `processSource` function call. Now check if a prediction was persisted by running the following fetch call: 
```js
c3Grid(SmartBulbPrediction.fetch({filter: 'smartBulb=="YOUR SOURCE INSTANCE ID"'}))
```

If all looks good and you see a new prediction, move on to method 2.

#### Method 2: 
Now provision your code and load your input parameters the same was as Method \#1 above. 

Then pass the input parameters into a direct call to the Analytic code provisioned in the platform. 

Debug as necessary. 


## Section 3 - Executing in Analytic Container Engine <a class="anchor" id="3"></a>

First, call `SmartBulbPrediction.removeAll()` in console to remove any existing predictions you have in the platform. Call `c3Count(SmartBulbPrediction)` to ensure all the predictions have been deleted. 

There are two ways to manually execute an Analytic. Try both: 

#### Method 1: `fireAnalytic` (synchronous)

With `startDate = "2011-01-01"` and `endDate = "2012-01-01"`, run:

```js
AnalyticsContainer.fireAnalytics(
      [{
        typeId: Tag.getTypeId('structure', 'SmartBulb'),
        objId: 'SMBLB1',
        timeRanges: [{start: startDate, end: endDate}]
      }],
      ["SmartBulbPredictionAnalytic"],
      {forceReEval: true}
    );
```

#### Method 2: `refreshAnalytics` (asynchronous)

**Step 1**: With the model, DFEs, and metrics you have persisted in the platform, run `refreshAnalytics()` to retroactively generate predictions on all historical data using the following syntax:

```js
var job = SmartBulb.refreshAnalytics({
    start: "2010-10-01",
    end: "2016-05-05",
    batchSize: 1,
    metricFailure: 'atEnd',
    forceReEval: true, 
	analyticWhitelist: ['SmartBulbPredictionAnalytic']
  });
```

**Step 2**: Use `c3Grid(Cluster.actionDump())` and `c3Grid(InvalidationQueue.countAll())` to monitor your jobs.
Note that ACE jobs submitted with the `refreshAnalytics` command go to the `BatchQueue`, instead of the `AnalyticsQueue`, as they normally would. 

**Step 3**: Debug with `c3QErrs(QueueName)` as necessary.


Now, if you fetch or count on `SmartBulbPrediction`, you should see new predictions for the dates between the start and end dates specified above.

## Section 4 -  Streaming Predictions <a class="anchor" id="4"></a>

You should now have a trained machine learning model that is hooked up to streaming capabilities using the **C3 Analytic Container Engine**, such that, when new data arrives, your **C3 Data Flow Events** will trigger business logic to fetch the latest persisted model **and** write predictions based on the new data.

Let's verify that this works using SmartBulb **2**.

Since this SmartBulb **has** failed, we are going to generate new data that follows the last data point we have, and verify that we have new predictions as a result of this new data coming in. To do so, follow the three steps below:

**Step 1**: Run the following command in a snippet to create new measurement data:
```js
 var meas = SmartBulbMeasurement.make({"id": "SBMS_serialNo_SMBLB2#TESTMEAS", "lumens": 1230, "power": 18,
            "temperature": 78, "voltage" : 125, "status": 1, "parent":"SBMS_serialNo_SMBLB2", "start": 
            "2018-12-31T08:00:00", "end": "2018-12-31T09:00:00" });

 SmartBulbMeasurement.merge(meas)
```
**Step 2**: Refresh your console page to get to get the latest data.

**Step 3**: Run a `fetch` command on the `SmarBulbPrediction` type, filtering on `SMBLB2` and ordering by `descending(timestamp)`.

**You should see a new prediction generated based on the data you just generated!**

## Conclusion

Congratulations! You have now successfully become more familiar with ACE. While doing this, you implemented various DFEs and properly handled them through Analytics!

## Make sure to CLOSE AND HALT  (from bottom of the File menu at the top) this notebook when you leave.

## Appendix - Kernel Installation Instructions <a class="anchor" id="apdx">
    
If the **`py-pythonEnv`** kernel is not available, perform the following:
1. Select the 'Kernel' tab at the top of this notebook, then select 'Manage Kernels';
2. Scroll to find `py-pythonEnv`, and click the 'Install' button;
> Note: This installation will "fail", but don't worry, exit the Manage Kernels UI and return to the main notebook.
3. Navigate to the 'File' tab located at the top of the notebook, and select 'Close and Hault'. This will close the notebook. Re-open it.
4. You should notice, in the upper-right corner, the kernel is now set correctly to `py-pythonEnv`. If it is not, navigate to the Kernel tab, then select 'Change Kernel' and select the appropriate option.